<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Waldo Campos Maldonado


### **Link de repositorio de GitHub:** `https://github.com/WaldoCampos/mds7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Introducción a procesamiento del lenguaje natural.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 26/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Visualizar el funcionamiento de clasificadores.
- Realizar una GridSearch sobre un conjunto de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [2]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Waldo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Waldo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [4]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
df_comics.columns

Index(['Unnamed: 0', 'name', 'real_name', 'full_name', 'overall_score',
       'history_text', 'powers_text', 'intelligence_score', 'strength_score',
       'speed_score', 'durability_score', 'power_score', 'combat_score',
       'superpowers', 'alter_egos', 'aliases', 'place_of_birth',
       'first_appearance', 'creator', 'alignment', 'occupation', 'base',
       'teams', 'relatives', 'gender', 'type_race', 'height', 'weight',
       'eye_color', 'hair_color', 'skin_color', 'img', 'has_electrokinesis',
       'has_energy_constructs', 'has_mind_control_resistance',
       'has_matter_manipulation', 'has_telepathy_resistance',
       'has_mind_control', 'has_enhanced_hearing', 'has_dimensional_travel',
       'has_element_control', 'has_size_changing', 'has_fire_resistance',
       'has_fire_control', 'has_dexterity', 'has_reality_warping',
       'has_illusions', 'has_energy_beams', 'has_peak_human_condition',
       'has_shapeshifting', 'has_heat_resistance', 'has_jump',
       'has_

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `bag of words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `bag of words`?...

`bag of words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `bag of words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `bag of words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [7]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [8]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Waldo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [9]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names())

C:\Users\Waldo\anaconda3\envs\mds7202\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names())

C:\Users\Waldo\anaconda3\envs\mds7202\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar MinMaxScaler sobre los atributos de interes.

**Respuesta:**

In [12]:
stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

bow1 = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(df_comics['history_text'])
print(df.shape)
bow2 = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(df_comics['history_text'])
print(df.shape)

(1285, 333920)
(1285, 333920)


In [13]:
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

df = MinMaxScaler().fit_transform(df_comics.loc[:, atributos_de_interes])
df.shape

(1285, 6)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**To-DO:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1 y aplicar un clasificador  `MultinomialNB()`.
- [ ] Entrenar el pipeline y comentar los resultados.

**Respuesta:**

In [14]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))

ct = ColumnTransformer([
    ('bow', bow, 'history_text'),
    ('minmaxscaler', MinMaxScaler(), atributos_de_interes)
])

bow_pipe = Pipeline([
    ('preprocessing', ct),
    ('feature selection', SelectPercentile(f_classif, percentile=90)),
    ('naive bayes', MultinomialNB())
])

In [15]:
X = df_comics.drop(columns=['alignment'])
Y = df_comics['alignment']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, shuffle=True, stratify=Y, random_state=69)

In [16]:
nb = bow_pipe.fit(X_train, Y_train)
y_pred_nb = nb.predict(X_test)

In [17]:
print(classification_report(Y_test, y_pred_nb))

              precision    recall  f1-score   support

         Bad       0.62      0.09      0.16       142
        Good       0.60      0.97      0.74       246
     Neutral       0.00      0.00      0.00        37

    accuracy                           0.59       425
   macro avg       0.41      0.35      0.30       425
weighted avg       0.55      0.59      0.48       425



In [18]:
y_pred_nb

array(['Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Bad', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Neutral', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Bad', 'Bad', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Neutral',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Bad', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Bad', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
     

Un 59% de las predicciones son correctas, sin embargo el modelo está prediciendo que la gran mayoría de los datos tienen la etiqueta 'Good', estos datos corresponden al 57% del dataset, por lo que un modelo que solo predice 'Good' ya tendría un 57% de accuracy. Esto significa que el modelo no es muy bueno identificando personajes de alinamiento malvado o neutral.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Dos clasificadores distintos en donde varie sus parámetros. Se le recomienda utilizar `LogisticRegression()` y `RandomForestClassifier()`.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. 
- Selección de las mejores columnas para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
        'clf__multi_class': ['liblinear']},
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [19]:
bow_pipe.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('bow',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x000001C571AB06D0>),
                                    'history_text'),
                                   ('minmaxscaler', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('feature selection', SelectPercentile(percentile=90)),
  ('naive bayes', MultinomialNB())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('bow',
                                  CountVectorizer(ngram_range=(1, 2),
                                                  tokenizer=<__main__.StemmerTokenizer object at 0x000001C571AB06D0>),
              

In [22]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

bow_pipe = Pipeline([
    ('preprocessing', ct),
    ('feature selection', SelectPercentile()),
    ('clf', None)
])

params = [
       # logistic regression
       {'preprocessing__bow__ngram_range': [(1,1), (1,2), (1,3)],
        'feature selection__percentile': [20, 40, 60 ,80],
        'feature selection__score_func': [f_classif],
        'clf': [LogisticRegression(random_state=420)],
        'clf__C': [0.1, 1, 10]},
       # random forest  
       {'preprocessing__bow__ngram_range': [(1,1), (1,2), (1,3)],
        'feature selection__percentile': [20, 40, 60 ,80],
        'feature selection__score_func': [f_classif],
        'clf': [RandomForestClassifier(random_state=69)],
        'clf__criterion': ['gini', 'entropy']}
       ]

search = HalvingGridSearchCV(bow_pipe, params, verbose=10, scoring='accuracy')

In [23]:
search.fit(X_train, Y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 31
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 60
n_resources: 31
Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5; 1/60] START clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=20, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1)
[CV 1/5; 1/60] END clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=20, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1);, score=(train=0.958, test=0.667) total time=   0.8s
[CV 2/5; 1/60] START clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=20, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1)
[CV 2/5; 1/60] END clf=L

[CV 2/5; 4/60] END clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=40, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.167) total time=   0.5s
[CV 3/5; 4/60] START clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=40, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1)


C:\Users\Waldo\anaconda3\envs\mds7202\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/5; 4/60] END clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=40, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1);, score=(train=0.958, test=0.167) total time=   0.9s
[CV 4/5; 4/60] START clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=40, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1)
[CV 4/5; 4/60] END clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=40, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.667) total time=   1.3s
[CV 5/5; 4/60] START clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=40, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1)
[CV 5/5; 4

C:\Users\Waldo\anaconda3\envs\mds7202\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

[CV 4/5; 3/3] END clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=80, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1);, score=(train=0.984, test=0.695) total time=  31.3s
[CV 5/5; 3/3] START clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=80, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1)


C:\Users\Waldo\anaconda3\envs\mds7202\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5; 3/3] END clf=LogisticRegression(random_state=420), clf__C=0.1, feature selection__percentile=80, feature selection__score_func=<function f_classif at 0x000001C56F3F41F0>, preprocessing__bow__ngram_range=(1, 1);, score=(train=0.985, test=0.689) total time=  33.6s


C:\Users\Waldo\anaconda3\envs\mds7202\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('bow',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x000001C571AB06D0>),
                                                                                'history_text'),
                                                                               ('minmaxscaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                   

In [25]:
y_pred_grid = search.predict(X_test)

In [26]:
print(classification_report(Y_test, y_pred_grid))

              precision    recall  f1-score   support

         Bad       0.66      0.54      0.59       142
        Good       0.73      0.89      0.80       246
     Neutral       0.00      0.00      0.00        37

    accuracy                           0.70       425
   macro avg       0.46      0.48      0.47       425
weighted avg       0.64      0.70      0.66       425



Los resultados mejoran significativamente, ahora se obtienen mejores resultados clasificando personajes malos. Aún se tiene malos resultados para personajes neutrales, pero esto puede ser explicado por el bajo soporte de esta clase.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [35]:
df_comics_no_label = pd.read_csv('comics_no_label.csv').dropna(subset=['history_text'])

In [36]:
df_comics_no_label

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
6,46,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
80,1396,Wendigo,Wendigo,NaN,6,The Wendigo are the result of an ancient curse...,NaN,65,80,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81,1431,XS (CW),Nora West-Allen,NaN,19,"According to Eobard Thawne, the daughter of Ba...",Nora possesses a natural connection to the Spe...,90,55,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
82,1440,Yondu (MCU),Yondu,NaN,7,"Yondu Udonta was a Centaurian, leader of a key...","Yondu Udonta was a member of the Centaurians, ...",85,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
search.predict(df_comics_no_label[df_comics_no_label['name'] == 'Vergil'])

array(['Good'], dtype=object)

In [42]:
search.predict(df_comics_no_label[df_comics_no_label['name'] == 'Gorilla Girl'])

array(['Good'], dtype=object)

In [43]:
search.predict(df_comics_no_label[df_comics_no_label['name'] == 'Batcow'])

array(['Good', 'Good'], dtype=object)

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>